In [ ]:
import datetime
import os
import glob
import math

import pandas as pd
from dateutil.parser import parse

from django.apps import apps
from django.db.utils import DataError
from django.core.exceptions import FieldDoesNotExist

In [ ]:
glob_pattern = r"R:\OREA-EGYPT_Puzzle4D\Metadaten\4DP_Metadaten_*.xlsx"

In [ ]:
files = glob.glob(glob_pattern)

In [ ]:
all_models = [x for x in apps.all_models['archiv'].values() if not '_' in x.__name__]

In [ ]:
len(all_models)

In [ ]:
file_class_map = {}
for cl in all_models:
    for file in files:
        con_file_name = f"4DP_Metadaten_{cl.__name__}.xlsx".lower()
        if con_file_name in file.lower():
            file_class_map[cl.__name__] = file

In [ ]:
file_class_map

In [ ]:
for current_class in all_models:
    model_name = current_class.__name__
    try:
        source_file = file_class_map[current_class.__name__]
    except KeyError:
        continue
    try:
        df_data = pd.read_excel(source_file)
        print(source_file)
    except FileNotFoundError:
        df_data = False
        continue
    if isinstance(df_data, pd.DataFrame):
        df_data.columns = map(str.lower, df_data.columns)
        df_keys = df_data.keys()
        nr_cols = len(df_keys)
        for i, row in df_data.iterrows():
            temp_item, _ = current_class.objects.get_or_create(legacy_id=row[0])
            col_counter = 0
            while col_counter < nr_cols:
                cur_attr = df_keys[col_counter]
                try:
                    cur_attr_type = current_class._meta.get_field(df_keys[col_counter]).get_internal_type()
                except FieldDoesNotExist:
                    cur_attr_type = None
                if cur_attr_type is not None:

                    if "{}".format(cur_attr_type) == "CharField":
                        try:
                            my_val = "{}".format(row[cur_attr])[:249]
                            setattr(temp_item, cur_attr, my_val)
                        except TypeError:
                            pass

                    elif "{}".format(cur_attr_type) == "ForeignKey" and isinstance(row[cur_attr], str):
                        fk  = current_class._meta.get_field(df_keys[col_counter])
                        rel_model_name = fk.related_model._meta.model_name
                        temp_rel_obj, _ = fk.related_model.objects.get_or_create(legacy_id=row[cur_attr])
                        if rel_model_name == 'skosconcept':
                            temp_rel_obj.pref_label = row[cur_attr]
                            col, _ = SkosCollection.objects.get_or_create(name=f"{df_keys[col_counter]}")
                            temp_rel_obj.collection.add(col)
                            temp_rel_obj.save()
                        setattr(temp_item, cur_attr, temp_rel_obj)

                    elif "{}".format(cur_attr_type) == "ManyToManyField" and isinstance(row[cur_attr], str):
                        fk = current_class._meta.get_field(df_keys[col_counter])
                        rel_model_name = fk.related_model._meta.model_name
                        rel_things = []
                        for x in row[cur_attr].split("|"):
                            temp_rel_obj, _ = fk.related_model.objects.get_or_create(legacy_id=x.strip())
                            rel_things.append(temp_rel_obj)
                            m2m_attr = getattr(temp_item, cur_attr)
                        m2m_attr.set(rel_things)
                    elif "{}".format(cur_attr_type) == "TextField" and isinstance(row[cur_attr], str):
                        setattr(temp_item, cur_attr, row[cur_attr])

                    elif "{}".format(cur_attr_type) == "DateField":
                        if isinstance(row[cur_attr], datetime.date):
                            value = row[cur_attr]
                        else:
                            try:
                                value = parse(row[cur_attr])
                            except Exception as e:
#                                 print(f"{row[cur_attr]} could not be parsed, due to Error: {e}")
                                value = None
                        if value is not None:
                            setattr(temp_item, cur_attr, value)
    
                    elif "{}".format(cur_attr_type) == "DecimalField":
                        try:
                            value = float(row[cur_attr])
                        except ValueError:
#                             print("{} can't be parsed as float".format(row[cur_attr]))
                            value = math.nan
                        if value is not None:
                            setattr(temp_item, cur_attr, value)
                temp_item.save()
                col_counter += 1
#                 print(col_counter)